In [1]:
import json
import string
import random
import nltk
nltk.download('omw-1.4')
import numpy as num
from nltk.stem import WordNetLemmatizer # It has the ability to lemmatize.
import tensorflow as tensorF # A multidimensional array of elements is represented by this symbol.
from tensorflow.keras import Sequential # Sequential groups a linear stack of layers into a tf.keras.Model
from tensorflow.keras.layers import Dense, Dropout

nltk.download("punkt")# required package for tokenization
nltk.download("wordnet")# word database

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
ourData = {
    "ourIntents":
     [

    {"tag": "greeting",
     "patterns": ["Hi", "Hello", "Hey"],
     "responses": ["Welcome to our online store! How may I assist you?", "Hello! How can I help you today?", "Hi there! How may we serve you?"],
    },

    {"tag": "product_inquiry",
     "patterns": ["Tell me about the products you offer", "What items are available?", "Show me your product catalog"],
     "responses": ["Explore our diverse product range, featuring electronics, clothing, accessories, and more. Feel free to browse our catalog on our website!"],
    },

    {"tag": "price_inquiry",
     "patterns": ["What is the cost?", "How much does it cost?", "Can you tell me the price?"],
     "responses": ["For the most up-to-date pricing information, please visit our website or reach out to our customer support. Prices may vary based on promotions and product availability."],
    },

    {"tag": "shipping_info",
     "patterns": ["Tell me about your shipping services", "How does the shipping process work?", "Provide information about delivery"],
     "responses": ["We offer reliable and timely shipping services. Shipping fees and delivery times may vary based on your location. Find more details during the checkout process."],
    },

    {"tag": "payment_info",
     "patterns": ["What are the payment options?", "Tell me about accepted payment methods", "How can I make a payment?"],
     "responses": ["We accept various payment methods, including credit cards, PayPal, and more. You can choose the payment option that best suits you during the checkout process."],
    },

    {"tag": "discounts",
     "patterns": ["Are there any discounts?", "Tell me about promotions", "Discount offers"],
     "responses": ["Yes, we frequently offer promotions and discounts. Check our website or subscribe to our newsletter to stay updated on the latest deals!"],
    },

    {"tag": "order_status",
     "patterns": ["Check my order status", "Where is my package?", "Track my order"],
     "responses": ["To check your order status, please log in to your account on our website. There, you can find real-time updates on your order and shipment."],
    },

    {"tag": "returns",
     "patterns": ["Can I return a product?", "What is your return policy?", "Return process"],
     "responses": ["We have a hassle-free return policy. If you're not satisfied with your purchase, you can initiate a return within [number of days] days of receiving the product. Visit our Returns page on the website for more details."],
    },

    {"tag": "contact_support",
     "patterns": ["How can I contact customer support?", "Need help", "Customer service contact"],
     "responses": ["Our customer support team is here to assist you. You can reach us through [email], our helpline at [phone number], or use the live chat feature on our website."],
    },

    {"tag": "order_history",
     "patterns": ["View my order history", "Previous orders", "Order records"],
     "responses": ["To view your order history, log in to your account and navigate to the Order History section. There, you can find a detailed record of all your past orders."],
    },
    {"tag": "goodbye",
     "patterns": ["bye", "see you later", "goodbye"],
     "responses": ["Thank you for visiting our online store! If you have any further questions, feel free to ask. Goodbye!", "Goodbye! Have a wonderful day!"],
    }

   ]
}



In [3]:
lm = WordNetLemmatizer() #for getting words
# lists
ourClasses = []
newWords = []
documentX = []
documentY = []
# Each intent is tokenized into words and the patterns and their associated tags are added to their respective lists.
for intent in ourData["ourIntents"]:
    for pattern in intent["patterns"]:
        ournewTkns = nltk.word_tokenize(pattern)# tokenize the patterns
        newWords.extend(ournewTkns)# extends the tokens
        documentX.append(pattern)
        documentY.append(intent["tag"])


    if intent["tag"] not in ourClasses:# add unexisting tags to their respective classes
        ourClasses.append(intent["tag"])

newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation] # set words to lowercase if not in punctuation
newWords = sorted(set(newWords))# sorting words
ourClasses = sorted(set(ourClasses))# sorting classes

In [4]:
trainingData = [] # training list array
outEmpty = [0] * len(ourClasses)
# bow model
for idx, doc in enumerate(documentX):
    bagOfwords = []
    text = lm.lemmatize(doc.lower())
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1
    trainingData.append([bagOfwords, outputRow])

random.shuffle(trainingData)
trainingData = num.array(trainingData, dtype=object)# coverting our data into an array afterv shuffling

x = num.array(list(trainingData[:, 0]))# first trainig phase
y = num.array(list(trainingData[:, 1]))# second training phase

In [5]:
iShape = (len(x[0]),)
oShape = len(y[0])
# parameter definition
ourNewModel = Sequential()
# In the case of a simple stack of layers, a Sequential model is appropriate

# Dense function adds an output layer
ourNewModel.add(Dense(128, input_shape=iShape, activation="relu"))
# The activation function in a neural network is in charge of converting the node's summed weighted input into activation of the node or output for the input in question
ourNewModel.add(Dropout(0.5))
# Dropout is used to enhance visual perception of input neurons
ourNewModel.add(Dense(64, activation="relu"))
ourNewModel.add(Dropout(0.3))
ourNewModel.add(Dense(oShape, activation = "softmax"))
# below is a callable that returns the value to be used with no arguments
md = tensorF.keras.optimizers.Adam(learning_rate=0.005)
# Below line improves the numerical stability and pushes the computation of the probability distribution into the categorical crossentropy loss function.
ourNewModel.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=["accuracy"])
# Output the model in summary
print(ourNewModel.summary())
# Whilst training your Nural Network, you have the option of making the output verbose or simple.
ourNewModel.fit(x, y, epochs=200, verbose=1)
# By epochs, we mean the number of times you repeat a training set.

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               8448      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 11)                715       
                                                                 
Total params: 17419 (68.04 KB)
Trainable params: 17419 (68.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/200
2/2

In [6]:
def ourText(text):
  newtkns = nltk.word_tokenize(text)
  newtkns = [lm.lemmatize(word) for word in newtkns]
  return newtkns

def wordBag(text, vocab):
  newtkns = ourText(text)
  bagOwords = [0] * len(vocab)
  for w in newtkns:
    for idx, word in enumerate(vocab):
      if word == w:
        bagOwords[idx] = 1
  return num.array(bagOwords)

def Pclass(text, vocab, labels):
  bagOwords = wordBag(text, vocab)
  ourResult = ourNewModel.predict(num.array([bagOwords]))[0]
  newThresh = 0.2
  yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

def getRes(firstlist, fJson):
  tag = firstlist[0]
  listOfIntents = fJson["ourIntents"]
  for i in listOfIntents:
    if i["tag"] == tag:
      ourResult = random.choice(i["responses"])
      break
  return ourResult

In [7]:
while True:
    newMessage = input("")

    # Tentukan kondisi untuk menghentikan loop
    if newMessage.lower() == "exit":
        break

    intents = Pclass(newMessage, newWords, ourClasses)
    ourResult = getRes(intents, ourData)
    print(ourResult)

hi
1/1 [==============================] - 0s 96ms/step
Welcome to our online store! How may I assist you?
what is the cost?
1/1 [==============================] - 0s 22ms/step
For the most up-to-date pricing information, please visit our website or reach out to our customer support. Prices may vary based on promotions and product availability.
bye
1/1 [==============================] - 0s 30ms/step
Goodbye! Have a wonderful day!
oke by
1/1 [==============================] - 0s 30ms/step
Hi there! How may we serve you?
exit
